## Importing the Libraries

In [71]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import mysql.connector as sql
from mysql.connector import Error
import csv

## Connection to MySQL Database

In [2]:
try:
    conn = sql.connect(host = 'localhost', user = 'root', password = 'The28Table')
    
    if conn.is_connected():
        c = conn.cursor()
        c.execute("CREATE DATABASE SportsStats")
    
except Error as e:
    print("Error while connecting to MySQL:", e)

Error while connecting to MySQL: 1007 (HY000): Can't create database 'sportsstats'; database exists


In [127]:
conn = sql.connect(host = 'localhost', user = 'root', password = 'The28Table', database = 'sportsstats')

c = conn.cursor(buffered = True)
c.execute('SELECT year, COUNT(*) AS participants\
                FROM athleteevent\
                GROUP BY year\
                ORDER BY year ASC')

result1 = c.fetchall()

In [128]:
Participation_per_year = pd.DataFrame(result1, columns = ['Year', 'Number of Participants'])
Participation_per_year

,Year,Number of Participants
0,1896,380
1,1900,1936
2,1904,1301
3,1906,1733
4,1908,3101
5,1912,4040
6,1920,4292
7,1924,5693
8,1928,5574
9,1932,3321


In [5]:
conn = sql.connect(host = 'localhost', user = 'root', password = 'The28Table', database = 'sportsstats')

c = conn.cursor(buffered = True)
c.execute('SELECT year, COUNT(DISTINCT event)\
                FROM athleteevent\
                GROUP BY year\
                ORDER BY year ASC')

result2 = c.fetchall()

In [124]:
Events_per_year = pd.DataFrame(result2, columns = ['Year', 'Number of Events'])
Events_per_year

,Year,Number of Events
0,1896,43
1,1900,90
2,1904,95
3,1906,74
4,1908,109
5,1912,107
6,1920,158
7,1924,148
8,1928,136
9,1932,145


In [7]:
conn = sql.connect(host = 'localhost', user = 'root', password = 'The28Table', database = 'sportsstats')

c = conn.cursor(buffered = True)
c.execute('SELECT year, COUNT(medal)\
                FROM athleteevent\
                WHERE medal <> "None"\
                GROUP BY year\
                ORDER BY year ASC')

result3 = c.fetchall()

In [123]:
Medals_per_year = pd.DataFrame(result3, columns = ['Year', 'Count of Medals'])
Medals_per_year

,Year,Count of Medals
0,1896,143
1,1900,604
2,1904,486
3,1906,458
4,1908,831
5,1912,941
6,1920,1308
7,1924,962
8,1928,823
9,1932,739


In [9]:
x_participants = Participation_per_year.iloc[:,1]
y_events = Events_per_year.iloc[:,1]
z_medals = Medals_per_year.iloc[:,1]

In [10]:
x_participants.head()

0     380
1    1936
2    1301
3    1733
4    3101
Name: Number of Participants, dtype: int64

In [11]:
y_events.head()

0     43
1     90
2     95
3     74
4    109
Name: Number of Events, dtype: int64

In [12]:
PCC1 = np.corrcoef(x_participants, y_events)
print(PCC1)

[[1.         0.95665494]
 [0.95665494 1.        ]]


In [13]:
PCC2 = np.corrcoef(x_participants, z_medals)
print(PCC2)

[[1.         0.93613907]
 [0.93613907 1.        ]]


In [14]:
PCC3 = np.corrcoef(y_events, z_medals)
print(PCC3)

[[1.         0.98635421]
 [0.98635421 1.        ]]


## Multiple relationships have been discovered amongst: 
### Number of participants - Number of Events - Count of Medals

#### Pearson Correlation Coefficient has been calculated between:
#### Number of Participants & Number of Events = 0.95
#### Number of Participants & Count of Medals = 0.93
#### Number of Events & Count of Medals = 0.98

#### Correlation coefficient quantified from all the above mentioned relations came out to be highly positive. Therefore, we can ascertain that a country if sends more number of participants taking part in more number of events, they stand a chance of winning more medals. 

## Linear Regression

In [122]:
year_table = pd.merge(Participation_per_year, Events_per_year, on = 'Year')
year_table1 = pd.merge(year_table, Medals_per_year, on = 'Year')
year_table2 = year_table1.drop('Year', axis = 'columns')
year_table2

,Number of Participants,Number of Events,Count of Medals
0,380,43,143
1,1936,90,604
2,1301,95,486
3,1733,74,458
4,3101,109,831
5,4040,107,941
6,4292,158,1308
7,5693,148,962
8,5574,136,823
9,3321,145,739


In [81]:
X = year_table2[['Number of Participants', 'Number of Events']]
Y = year_table2[['Count of Medals']]

In [82]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.20)

In [83]:
model = LinearRegression()
model.fit(X_train, Y_train)

LinearRegression()

In [84]:
pred=model.predict(X_test)

In [85]:
pred[0:5]

array([[1336.69728294],
       [ 261.65027919],
       [1998.46405785],
       [ 609.31532572],
       [1138.70572777]])

In [90]:
model.predict([[3000, 145]])

array([[972.18869777]])

In [67]:
model.score(X,Y)

0.9734105689897747

#### Through this regression, we can see the accuracy comes out to be 97%. That means, this model can indeed be referred by countries to predict if 'n' number of participants are sent to participate in 'm' number of events, then they can expect the number of medals coming across their way.


##### Looking through the descriptive stats we can gauge how some variables are so inter-linked with each other that they can establish a strong relationship between them. This indeed can help the intended stakeholders determine how the events and the participation have an impact on winning the medals.